## RQ1 Creating FAO forest map

[Add Description]

Need to create a FAO Definition Approximation following the steps from Johnson et al (2023)

Steps:
1. Create inverse raster of GER LULC Class 3 & 4
2. Subtract inverse raster from JAXA FNF
3. Combine clipped JAXA FNF with GER LULC FNF
4. Copy & rename

In [3]:
# SETUP

# Note: this .ipynb file depends on files & folder structures created in rq1_step1_data_prep.ipynb

# Import packages
import glob
import os
import subprocess
import shutil

import pandas as pd
import geopandas as gpd
import rasterio

# Store gdal.exe paths
gdal_rasterize = "./thesis_env_conda/Library/bin/gdal_rasterize.exe"
gdalwarp = "./thesis_env_conda/Library/bin/gdalwarp.exe"

# Store gdal.py paths
gdal_calc = "./thesis_env_conda/Lib/site-packages/GDAL-3.10.1-py3.12-win-amd64.egg-info/scripts/gdal_calc.py"


### Step 1: Create inverse raster of GER LULC Class 3 & 4

Originally I planned to use a vector of the GER LULC Class 3 and 4 as a clipper to extract only those areas from the JAXA FNF raster. However, this resulted in a very complex vector and the clipping process was going to be prohibitively slow. 

Instead I decided to rasterise the GER LULC 3 and 4 Classes and then create an inverse raster where Class 3 & 4 = 0 and everything else = 1. I can then use this inverse raster to subtract the non-class-3 and non-class-4 areas from the JAXA FNF map. 

Note the dissolve and explode steps are not totally needed (previously they were required for creating a clipper shp).

In [2]:
# 1: COMBINE GER LULC SHPS
# Takes about 12 min

# Load all the GER LULC SHPs (with no columns - these are not needed)
ger_lulc_class3_shp = gpd.read_file("./processing/clc5_class3xx_3035_DE.shp", columns = [""])
ger_lulc_class4_shp = gpd.read_file("./processing/clc5_class4xx_3035_DE.shp", columns = [""])

# Append the shapefiles together
merged_ger_lulc_shp = pd.concat([ger_lulc_class3_shp,
                                 ger_lulc_class4_shp,
                                 ])

# Dissolve the geometries (this creates one multi-part geometry)
dissolved_ger_lulc_shp = merged_ger_lulc_shp.dissolve()

# Explode the multi-part geometry into multiple single geometries
exploded_ger_lulc_shp = dissolved_ger_lulc_shp.explode()

# Write the merged, dissolved & exploded output to file
exploded_ger_lulc_shp.to_file('./processing/clc5_class3_class4_3035_DE.shp')


c:\Users\ninam\Documents\UZH\04_Thesis\code\nm_forest_thesis\thesis_env_conda\Lib\site-packages\pyogrio\core.py:35: RuntimeWarning: Could not detect GDAL data files.  Set GDAL_DATA environment variable to the correct path.
  _init_gdal_data()


In [ ]:
# 1: INVERSE RASTERISATION

# Store path to shp for rasterising
gerlulc_3_4_shp = "./processing/clc5_class3_class4_3035_DE.shp"

# Store path/filename for inverse rasterised output
gerlulc_3_4_inverse = "./processing/clc5_class3_class4_3035_DE_5m_inverse.tif"

# Run gdal_rasterize to create inverse GER LULC Class 3 & 4 raster
# '-i' flag: invert rasterisation (burn value is burned into all parts NOT inside the polygons)
inverse_rasterise = subprocess.run([gdal_rasterize, 
                                    '-l', 'clc5_class3_class4_3035_DE',
                                    '-burn', '1',
                                    '-i',    
                                    '-tr', '5', '5',
                                    '-a_nodata', '-9999', 
                                    '-ot', 'Int16', 
                                    '-of', 'GTiff',
                                    '-co', 'COMPRESS=LZW', 
                                    '-co', 'BIGTIFF=YES', 
                                    gerlulc_3_4_shp,
                                    gerlulc_3_4_inverse
                                    ],
                                    capture_output=True, 
                                    text=True)

print(inverse_rasterise.stdout)
print(inverse_rasterise.stderr)


### Step 2: Subtract inverse raster from JAXA FNF

Using the inverse raster from step 1, the next step is to adjust the JAXA FNF map so that all non-class-3 and non-class-4 forests are removed. Essentially, this removes any areas of forest in the JAXA map which are in predominantly agricultural or urban areas.

Before I subtract the two layers, the extents will need to match - so here I follow a similar process as in "rq1_step1_data_prep.ipynb" to clip and adjust the extents of the inverse raster.

In [ ]:
# 2: CLIP TO CORINE BBOX

# Store path to CORINE bbox shp (created in "rq1_step1_data_prep.ipynb")
corine_bbox_shp = "./processing/corine_reclass_bbox.shp"

# Define function that clips tifs to the CORINE bbox (copy from "rq1_step1_data_prep.ipynb")
def bbox_clip(input_paths):
    # Iterate through the paths 
    for path in input_paths:
        # For output file naming: extract the input file name (with extension)
        name_w_ext = os.path.split(path)[1] 
        # For output file naming: remove extension
        root_name = name_w_ext[:-4]
        # For output file naming: assemble the new file path for the output
        output_path = "./processing/" + root_name + "_bboxclip.tif"

        # Run warp to crop to the CORINE bbox
        clip_to_bbox = subprocess.run([gdalwarp, 
                                       '-crop_to_cutline', 
                                       '-cutline', corine_bbox_shp, 
                                       '-tr', '5', '5',
                                       '-dstnodata', '-9999', 
                                       '-ot', 'Int16', 
                                       '-co', 'COMPRESS=LZW', 
                                       '-co', 'BIGTIFF=YES', 
                                       path, 
                                       output_path
                                       ],
                                       capture_output=True, 
                                       text=True)
        print(clip_to_bbox.stdout)
        print(clip_to_bbox.stderr)

# Run just for the single raster
bbox_clip(["./processing/clc5_class3_class4_3035_DE_5m_inverse.tif"])

In [ ]:
# 2: WARP EXTENTS

# First, extract the extents from the CORINE data (created in "rq1_step1_data_prep.ipynb")
corine_ref = rasterio.open("./processing/U2018_CLC2018_V2020_3035_DE_5m_bboxclip.tif")
corine_bounds  = corine_ref.bounds

# Store the bounds in the format required for gdalwarp
corine_xmin = str(corine_bounds[0])       # xmin = left
corine_ymin = str(corine_bounds[1])       # ymin = bottom
corine_xmax = str(corine_bounds[2])       # xmax = right
corine_ymax = str(corine_bounds[3])       # ymax = top


# Define function that warps rasters to the CORINE extents (copy from "rq1_step1_data_prep.ipynb")
def corine_warp(input_paths):
    # Iterate through the paths 
    for path in input_paths:
        # For output file naming: extract the input file name (with extension)
        name_w_ext = os.path.split(path)[1] 
        # For output file naming: remove extension from input file name
        name_wo_ext = os.path.splitext(name_w_ext)[0]
        # For output file naming: assemble the new file path for the output
        output_path = "./processing/" + name_wo_ext + "_warp_exts.tif"
        
        # Run warp to match all rasters to CORINE extents
        warp_extents = subprocess.run([gdalwarp, 
                                      '-t_srs', 'EPSG:3035', 
                                      #'-tr', '5', '5',
                                      '-te', corine_xmin, corine_ymin, corine_xmax, corine_ymax,
                                      #'-tap',
                                      '-ot', 'Int16', 
                                      '-co', 'COMPRESS=LZW', 
                                      '-co', 'BIGTIFF=YES', 
                                      path, 
                                      output_path
                                      ],
                                      capture_output=True, 
                                      text=True)
        print(warp_extents.stdout)
        print(warp_extents.stderr)


# Separate Hansen processing
corine_warp(["./processing/clc5_class3_class4_3035_DE_5m_inverse_warp_exts.tif"])

In [ ]:
# 2: CLIP TO CORINE FOOTPRINT

In [ ]:
# 2: SUBTRACT INVERSE RASTER

# Store paths to the two input maps
jaxa_FNF = "./outputs/jaxa_FNF_3035_DE_5m_FNF.tif"
ger_lulc_inverse = ""

# Runs gdal_calc.py to subtract the input rasters  
adjusted_jaxa = subprocess.run(['python', 
                                gdal_calc, 
                                '-A', jaxa_FNF, 
                                '-B', ger_lulc_inverse, 
                                '--outfile=./processing/jaxa_3035_DE_5m_adjusted.tif', 
                                '--calc=A-B', 
                                '--co=COMPRESS=LZW', 
                                '--co=BIGTIFF=YES', 
                                '--NoDataValue=-9999'
                                ],
                                capture_output=True, 
                                text=True)

print(adjusted_jaxa.stdout)
print(adjusted_jaxa.stderr)

### Step 3: Combine adjusted JAXA FNF with GER LULC FNF

The JAXA map now meets the FAO requirements so it can be added together with the GER LULC FNF map (which also is within the FAO definition thresholds).

The output from adding the two maps together will include values 0 to 2, so the final step is to convert the map back into a FNF output (with only values of 0 and 1).

In [ ]:
# 3: COMBINE ADJUSTED JAXA & GER LULC

# Store paths to the two input maps
adjusted_jaxa_FNF = "./processing/jaxa_3035_DE_5m_adjusted.tif"
ger_lulc_FNF = "./outputs/clc5_class3xx_3035_DE_5m_FNF.tif"

# Runs gdal_calc.py to add the input rasters together 
initial_fao = subprocess.run(['python', 
                              gdal_calc, 
                              '-A', adjusted_jaxa_FNF, 
                              '-B', ger_lulc_FNF, 
                              '--outfile=./processing/fao_3035_DE_5m_calc.tif', 
                              '--calc=A+B', 
                              '--co=COMPRESS=LZW', 
                              '--co=BIGTIFF=YES', 
                              '--NoDataValue=-9999'
                              ],
                              capture_output=True, 
                              text=True)

print(initial_fao.stdout)
print(initial_fao.stderr)

In [ ]:
# 3: RECLASSIFY TO FNF

# Store path to initial FAO map
initial_fao = "./processing/fao_3035_DE_5m_calc.tif"

# Runs gdal_calc.py in order to reclassify to FNF
reclass_fao = subprocess.run(['python', 
                              gdal_calc, 
                              '-A', initial_fao, 
                              '--outfile=./processing/fao_3035_DE_5m_calc_reclass.tif', 
                              '--calc=-9999*(A==-9999)+0*(A==0)+1*(A==1)+1*(A==2)', 
                              '--co=COMPRESS=LZW', 
                              '--co=BIGTIFF=YES', 
                              '--NoDataValue=-9999'
                              ],
                              capture_output=True, 
                              text=True)

print(reclass_fao.stdout)
print(reclass_fao.stderr)

### Step 4: Copy & Rename

After visually checking the rasters in QGIS, the outputs from the last step seem to meet all the requirments! I now copy over the raster to the "outputs" folder and rename it to indicate it is the FNF output. 

In [ ]:
# 4: COPY & RENAME

# Store the path to the old version (to be copied & renamed)
fao_old = "./processing/fao_3035_DE_5m_calc_reclass.tif"

# Copy & rename the raster
shutil.copy(fao_old, "./outputs/" + os.path.split(fao_old)[1][:-16] + "FNF.tif")